# Bayesian linear and GP regression

## Load the NOAA CO2 file

In [146]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read all lines, then drop comment lines starting with '#'
df = pd.read_fwf(
    "/work/scj-gdrive/co2.txt",
    widths=[6, 6, 12, 10, 10],
    header=None,
    comment="#",
    names=["year", "month", "t", "y", "trend"]
)

# Drop any non-numeric rows (e.g. leftover 'year month decimal ...' line)
df = df[pd.to_numeric(df["t"], errors="coerce").notna()].copy()
df["t"] = df["t"].astype(float)
df["y"] = df["y"].astype(float)

# Time and data
t = df["t"].values
y = df["y"].values

# Training mask and splits
mask_train = t <= 2007 + 8/12         # shape (N,)
t_train = t[mask_train]               # shape (N_train,)

## Mean and covariance

In [149]:
Phi = np.c_[t,np.ones(len(t))]
m0, S0 = np.array([0.,360.]), np.diag([1e4,1e4])
beta = 1.0
SN=np.linalg.inv(np.linalg.inv(S0)+beta*Phi.T@Phi) 
mN=SN@(np.linalg.inv(S0)@m0 + beta*Phi.T@y)

print('Mean:',mN)
print('Cov:',SN)
print('Std:',np.sqrt(np.diag(SN)))

## MAP estimates 

In [152]:
sigma2 = 1.0
beta = 1.0 / sigma2

a_MAP, b_MAP = mN
print("a_MAP:", a_MAP)
print("b_MAP:", b_MAP)

# Compute residuals g_obs(t)
g_obs = y - (a_MAP * t + b_MAP)
g_train = g_obs[mask_train]               # shape (N_train,)

# Quick diagnostics (optional)
print("Residual mean:", np.mean(g_obs))
print("Residual std:", np.std(g_obs))

# Plot residuals over time
plt.figure(figsize=(10, 4))
plt.plot(df["t"].values, g_obs, "b-", markersize=3)
plt.axhline(0.0, color="r", linestyle="--", linewidth=1)
plt.xlabel("Calendar time (decimal year)")
plt.ylabel("Residual g_obs(t) [ppm]")
plt.title("CO2 residuals g_obs(t) = y(t) - (a_MAP t + b_MAP)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [161]:
from scipy.stats import norm

res = g_train  # or g_obs

# histogram of residuals
plt.figure(figsize=(6, 4))
count, bins, _ = plt.hist(
    res,
    bins=30,
    density=True,
    alpha=0.7,
    color="C0",
    label="residuals",
)

# grid for normal pdf
xs = np.linspace(bins[0], bins[-1], 400)

# standard normal N(0,1); or fit mean/std of residuals instead
pdf_std = norm.pdf(xs, loc=0.0, scale=1.0)
plt.plot(xs, pdf_std, "r-", linewidth=2, label="e(t) ~ N(0,1)")

plt.xlabel("residual bin")
plt.ylabel("density")
plt.title("CO2 Residuals Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Draw samples from a GP

In [22]:
def sample_gp(x, kernel, params, jitter=1e-6, random_state=None):
    """
    Draw one sample f(x) from a zero-mean GP with covariance kernel k.

    Parameters
    ----------
    x : array_like, shape (N,)
        Input points where the function is evaluated.
    kernel : callable
        Covariance function k(s, t, *kernel_params) -> scalar.
    *kernel_params :
        Hyperparameters passed to the kernel.
    jitter : float, optional
        Small diagonal term for numerical stability.
    random_state : int or None, optional
        Seed for reproducibility.

    Returns
    -------
    f : ndarray, shape (N,)
        Sampled function values f(x) at the given inputs.
    """
    x = np.asarray(x)
    N = x.shape[0]

    # build covariance matrix K_ij = k(x_i, x_j)
    K = kernel(x, x, **params)
    
    # add jitter to ensure positive definiteness
    K += jitter * np.eye(N)

    rng = np.random.default_rng(random_state)
    f = rng.multivariate_normal(mean=np.zeros(N), cov=K)

    return f

### Samples from example RBF kernel

In [28]:
def rbf_kernel(s, t, lengthscale, variance):
    s = np.asarray(s)
    t = np.asarray(t)
    S, T = np.meshgrid(s, t, indexing="ij")  # both (N,M)
    diff = S - T
    return variance * np.exp(-0.5 * diff**2 / lengthscale**2)

x = np.linspace(0, 10, 200)

params1 = dict(lengthscale=1.0, variance=1.0)
params2 = dict(lengthscale=1.0, variance=1.0)
params3 = dict(lengthscale=0.3, variance=1.0)  # shorter lengthscale

f1 = sample_gp(x, rbf_kernel, params1, random_state=0)
f2 = sample_gp(x, rbf_kernel, params2, random_state=1)
f3 = sample_gp(x, rbf_kernel, params3, random_state=2)

plt.figure(figsize=(8, 4))
plt.plot(x, f1, label="ℓ = 1.0, sample 1")
plt.plot(x, f2, label="ℓ = 1.0, sample 2")
plt.plot(x, f3, label="ℓ = 0.3, sample 3")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title("Samples from a zero-mean GP (example kernel)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

### Samples from matern periodic kernel

In [10]:
def kst(s, t, theta, tau, sigma, phi, eta, zeta):
    """
    s, t : arrays of shape (N,) and (M,)
    returns K of shape (N, M)
    """
    s = np.asarray(s)
    t = np.asarray(t)

    # pairwise differences
    S, T = np.meshgrid(s, t, indexing="ij") # S, T have shape (N, M)
    diff = S - T

    # periodic part
    periodic = np.exp(-2 * np.sin(np.pi * diff / tau) ** 2 / sigma ** 2)
    # squared‑exponential part
    se = phi**2 * np.exp(-diff**2 / (2 * eta**2))
    
    # nugget
    K = theta**2 * (periodic + se)

    if zeta != 0.0:
        N = s.shape[0]
        if s.shape[0] == t.shape[0] and np.allclose(s, t):
            K = K + zeta**2 * np.eye(s.shape[0])
    return K

In [31]:
xs = np.linspace(0, 10, 300)

param_sets = [
    dict(theta=1.0, sigma=0.7, tau=1.0, phi=0.5, eta=2.0, zeta=0.0),
    dict(theta=1.0, sigma=0.2, tau=1.0, phi=0.5, eta=2.0, zeta=0.0),  # sharper periodic
    dict(theta=1.0, sigma=0.7, tau=2.0, phi=0.5, eta=2.0, zeta=0.0),  # longer period
    dict(theta=1.0, sigma=0.7, tau=1.0, phi=1.5, eta=0.5, zeta=0.0),  # more local SE
]

plt.figure(figsize=(10, 8))

for i, params in enumerate(param_sets, 1):
    f = sample_gp(xs, kst, params, random_state= i)
    plt.subplot(len(param_sets), 1, i)
    plt.plot(xs, f)
    plt.grid(True)
    plt.ylabel("f(t)")
    txt = ", ".join(f"{k}={v}" for k, v in params.items())
    plt.title("Samples from matern periodic kernel, hyperparameters:" + txt)

plt.xlabel("t")
plt.tight_layout()
plt.show()

## Test hyperparameters

### Minimise the negative log marginal likelihood to optimise hyperparameters

In [1]:
def nlml(p, t_train, g_train, jitter=1e-6):
    log_theta, log_tau, log_sigma, log_phi, log_eta, log_zeta = p

    # build params dict correctly
    params = dict(
        theta=np.exp(log_theta),
        tau=np.exp(log_tau),
        sigma=np.exp(log_sigma),
        phi=np.exp(log_phi),
        eta=np.exp(log_eta),
        zeta=np.exp(log_zeta),
    )

    # kst must have signature kst(s, t, **params)
    K = kst(t_train, t_train, **params)
    K = K + jitter * np.eye(len(t_train))

    # Cholesky for stability: K = L L^T
    L = np.linalg.cholesky(K)
    # Solve K^{-1} g via two triangular solves
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, g_train))

    data_fit = 0.5 * g_train @ alpha
    complexity = np.sum(np.log(np.diag(L)))  # 0.5 * log|K|
    const = 0.5 * len(t_train) * np.log(2 * np.pi)
    return data_fit + complexity + const

from scipy.optimize import minimize   # for minimising the negative log marginal likelihood

# initial guess in log-space (hand-tuned)
p0 = np.log([2.0, 1.0, 0.5, 0.5, 3.0, 0.4])  # [theta, tau, sigma, phi, eta, zeta]

res = minimize(
    nlml,
    p0,
    args=(t_train, g_train),
    method="L-BFGS-B",
    options={"maxiter": 100}
)

p_opt = res.x
theta_opt, tau_opt, sigma_opt, phi_opt, eta_opt, zeta_opt = np.exp(p_opt)
print("Optimized hyperparameters:")
print("theta, tau, sigma, phi, eta, zeta =", theta_opt, tau_opt, sigma_opt, phi_opt, eta_opt, zeta_opt)

Optimized hyperparameters:
theta, tau, sigma, phi, eta, zeta = 2.589086891839971 0.9990816088980068 1.7718435610262389 0.6563166765210973 1.2616660220940161 0.33558552328185726

In [34]:
params_opt = dict(
    theta=2.589086891839971,
    tau=0.9990816088980068,
    sigma=1.7718435610262389,
    phi=0.6563166765210973,
    eta=1.2616660220940161,
    zeta=0.33558552328185726,
)

# GP prior samples
fprior1 = sample_gp(t_train, kst, params_opt, random_state=0)
fprior2 = sample_gp(t_train, kst, params_opt, random_state=1)
fprior3 = sample_gp(t_train, kst, params_opt, random_state=2)

plt.figure(figsize=(10, 4))

# residuals
plt.plot(t_train, g_train, "b.", markersize=3, label="residuals $g(t)$")

# GP prior sample paths
plt.plot(t_train, fprior1, "g-", alpha=0.8, label="GP prior sample 1")
plt.plot(t_train, fprior2, "c-", alpha=0.8, label="GP prior sample 2")
plt.plot(t_train, fprior3, "m-", alpha=0.8, label="GP prior sample 3")

plt.xlabel("time $t$")
plt.ylabel("$g(t)$ / $f(t)$")
plt.title("Residuals vs GP prior samples with optimised hyperparameters")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Extrapolation

In [37]:
def fit_gp_residuals(t_train, g_train, theta, tau, sigma, phi, eta, zeta, jitter=1e-6):
    """Precompute training covariance inverse etc. for residual GP."""
    K = kst(t_train, t_train, theta, tau, sigma, phi, eta, zeta)
    K = K + jitter * np.eye(len(t_train))
    K_inv = np.linalg.inv(K)
    alpha = K_inv @ g_train  # used for predictive mean
    return {"theta": theta, "tau": tau, "sigma": sigma,
            "phi": phi, "eta": eta, "zeta": zeta,
            "t_train": t_train, "K_inv": K_inv, "alpha": alpha}

def gp_mean(t_test, gp_params):
    """Predictive mean of residual g(t) at test points."""
    th = gp_params
    t_train = th["t_train"]
    K_star = kst(t_test, t_train,
                     th["theta"], th["tau"], th["sigma"],
                     th["phi"], th["eta"], th["zeta"])
    # m_g = K_* K^{-1} g = K_* alpha
    return K_star @ th["alpha"]

def gp_std(t_test, gp_params):
    """Predictive std of residual g(t) at test points."""
    th = gp_params
    t_train = th["t_train"]
    K_inv = th["K_inv"]

    K_star = kst(t_test, t_train,
                     th["theta"], th["tau"], th["sigma"],
                     th["phi"], th["eta"], th["zeta"])
    K_starstar = kst(t_test, t_test,
                         th["theta"], th["tau"], th["sigma"],
                         th["phi"], th["eta"], th["zeta"])
    # Σ_g = K** - K_* K^{-1} K_*^T
    cov = K_starstar - K_star @ K_inv @ K_star.T
    var = np.clip(np.diag(cov), 0.0, np.inf)   # numerical safety
    return np.sqrt(var)

In [40]:
# Define test times: Sept 2007 to Dec 2020 (monthly)
t_test = np.arange(2007 + 8/12, 2020 + 12/12 + 1e-6, 1/12)

# Compute predictive mean / std of residual g(t) at t_test
gp_params = fit_gp_residuals(t_train, g_train,
                             theta_opt, tau_opt, sigma_opt, phi_opt, eta_opt, zeta_opt)
m_g = gp_mean(t_test, gp_params)
s_g = gp_std(t_test, gp_params)

# Turn residual predictions into CO2 predictions
f_mean = a_MAP * t_test + b_MAP + m_g
f_std  = s_g                   # var[f] = var[g] under this model

# Plot observed CO2 and extrapolated mean ± 1 std
plt.figure(figsize=(10, 4))
plt.plot(t, y, "m-", markersize=2, label="Observed CO$_2$")

# extrapolated GP mean
plt.plot(t_test, f_mean, "C0-", label="GP extrapolated mean")

# 1 std error bars as a shaded band
plt.fill_between(
    t_test,
    f_mean - f_std,
    f_mean + f_std,
    color="C0",
    alpha=0.2,
    label="±1 std (GP)"
)

plt.xlabel("Year")
plt.ylabel("CO$_2$ concentration (ppm)")
plt.title("CO$_2$ with linear trend + GP residual extrapolation")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [12]:
# =========================
# Define untrained vs trained hyperparameters
# =========================
# hand-chosen "untrained" values
theta_u, tau_u, sigma_u, phi_u, eta_u, zeta_u = 2.0, 1.0, 0.5, 0.5, 3.0, 0.4

# "trained" values = optimised hyperparameters

# =========================
# Fit GP for untrained and trained cases
# =========================
gp_un = fit_gp_residuals(t_train, g_train,
                         theta_u, tau_u, sigma_u, phi_u, eta_u, zeta_u)
gp_tr = fit_gp_residuals(t_train, g_train,
                         theta_opt, tau_opt, sigma_opt, phi_opt, eta_opt, zeta_opt)

# Predict residuals at test times
m_g_un = gp_mean(t_test, gp_un)
s_g_un = gp_std(t_test, gp_un)

m_g_tr = gp_mean(t_test, gp_tr)
s_g_tr = gp_std(t_test, gp_tr)

# Convert to CO2
f_un = a_MAP * t_test + b_MAP + m_g_un
f_tr = a_MAP * t_test + b_MAP + m_g_tr

# =========================
# Plot: trained vs untrained extrapolation in one block
# =========================
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

# LEFT: untrained hyperparameters
ax = axes[0]
ax.plot(t, y, "b.", ms=2, label="Observed CO$_2$")
ax.plot(t_test, f_un, "g-", label="Untrained mean")
ax.fill_between(t_test, f_un - s_g_un, f_un + s_g_un,
                color="C0", alpha=0.2, label="±1 std")
ax.set_title("Untrained hyperparameters")
ax.set_xlabel("Year")
ax.set_ylabel("CO$_2$ (ppm)")
ax.grid(True)
ax.legend()

# RIGHT: trained hyperparameters
ax = axes[1]
ax.plot(t, y, "b.", ms=2, label="Observed CO$_2$")
ax.plot(t_test, f_tr, "m-", label="Trained mean")
ax.fill_between(t_test, f_tr - s_g_tr, f_tr + s_g_tr,
                color="C1", alpha=0.2, label="±1 std")
ax.set_title("Trained hyperparameters")
ax.set_xlabel("Year")
ax.grid(True)
ax.legend()

plt.tight_layout()
plt.show()

# Mean-field learning

## E-Step

In [70]:
def MeanField(X, mu, sigma2, pi, lambda0, maxsteps=100, eps=1e-6):
    """
    Mean-field variational E-step for the binary latent factor model.

    X       : (N,D) data matrix
    mu      : (K,D) matrix, rows μ_k^T
    sigma2  : scalar noise variance
    pi      : (K,) Bernoulli priors π_k
    lambda0 : (N,K) initial λ_{nk}
    maxsteps: maximum iterations
    eps     : convergence threshold on |F_t - F_{t-1}|

    Returns
    -------
    lam : (N,K) mean-field parameters λ_{nk}
    F   : (T,) free energy values per iteration
    """
    X = np.asarray(X)
    N, D = X.shape

    mu = np.asarray(mu)
    if mu.shape[0] != X.shape[1] and mu.shape[1] == X.shape[1]:
        mu = mu  # assume (K,D)
    elif mu.shape[0] == X.shape[1] and mu.shape[1] != X.shape[1]:
        mu = mu.T  # convert (D,K) → (K,D)
    K = mu.shape[0]

    pi = np.asarray(pi).reshape(-1)
    assert pi.shape[0] == K

    lam = np.array(lambda0, dtype=float)
    assert lam.shape == (N, K)

    # Clip pi away from {0,1} before taking logs
    pi = np.clip(pi, 1e-3, 1-1e-3)
    logit_pi = np.log(pi) - np.log(1.0 - pi)

    F_vals = []

    for it in range(maxsteps):
        # ----- update λ for all data points -----
        for n in range(N):
            x_n = X[n]          # (D,)
            lam_n = lam[n]      # (K,)

            # precompute Σ_k λ_{nk} μ_k
            sum_lam_mu = lam_n @ mu  # (D,)

            new_lam_n = np.empty_like(lam_n)

            for k in range(K):
                mu_k = mu[k]               # (D,)
                sum_other = sum_lam_mu - lam_n[k] * mu_k

                # your derived term:
                # μ_k^T ( x_n - μ_k/2 - Σ_{k'≠k} λ_{nk'} μ_{k'} ) / σ²
                term = (mu_k @ (x_n - 0.5 * mu_k - sum_other)) / sigma2
                raw_logit = term + logit_pi[k]
                logit = np.clip(raw_logit, -20, 20)   # much tighter
                new_lam_n[k] = 1.0 / (1.0 + np.exp(-logit))

            lam[n] = new_lam_n

        # ----- compute free energy F(q,θ) using your expression -----
        # common precomputes
        mu_T = mu.T              # (D,K)
        # E[s_n] = λ_n, so E[sum_k s_nk μ_k] = lam @ mu
        M = lam @ mu             # (N,D)

        # first big bracket in F
        xTx = np.sum(X * X)      # ∑_n x_n^T x_n
        xT_M = np.sum(X * M)     # ∑_n x_n^T M_n
        # double sum over k,k'
        lam_mu = lam @ mu        # same as M
        # ||∑_k λ_nk μ_k||² = M_n^T M_n
        MM = np.sum(M * M)       # ∑_n M_n^T M_n

        const1 = -0.5 * D * X.shape[0] * np.log(2 * np.pi * sigma2)
        quad = -0.5 / sigma2 * (xTx - 2 * xT_M + MM)

        # prior term ∑_{n,k} [ λ_{nk} log π_k + (1-λ_{nk}) log(1-π_k) ]
        log_pi = np.log(pi + 1e-12)
        log_1m_pi = np.log(1 - pi + 1e-12)
        prior = np.sum(lam * log_pi + (1 - lam) * log_1m_pi)

        # entropy term −∑_{n,k} [ λ_{nk} log λ_{nk} + (1-λ_{nk}) log(1-λ_{nk}) ]
        log_lam = np.log(lam + 1e-12)
        log_1m_lam = np.log(1 - lam + 1e-12)
        entropy = -np.sum(lam * log_lam + (1 - lam) * log_1m_lam)

        F = const1 + quad + prior + entropy
        F_vals.append(F)
        
        # convergence criterion
        if it > 0 and abs(F_vals[-1] - F_vals[-2]) < eps:
            break

    return lam, np.array(F_vals)

## M-Step

In [73]:
def MStep(X, ES, ESS, eps=1e-6):
    """
    mu, sigma, pie = MStep(X,ES,ESS)

    Inputs:
    -----------------
           X: shape (N, D) data matrix
          ES: shape (N, K) E_q[s]
         ESS: shape (K, K) sum over data points of E_q[ss'] (N, K, K)
                           if E_q[ss'] is provided, the sum over N is done for you.

    Outputs:
    --------
          mu: shape (D, K) matrix of means in p(y|{s_i},mu,sigma)
       sigma: shape (,)    standard deviation in same
         pi: shape (1, K) vector of parameters specifying generative distribution for s
    """
    N, D = X.shape
    if ES.shape[0] != N:
        raise TypeError("ES must have the same number of rows as X")
    K = ES.shape[1]

    if ESS.shape == (N, K, K):
        ESS = np.sum(ESS, axis=0)
    if ESS.shape != (K, K):
        raise TypeError("ESS must be square and have the same number of columns as ES")

    # Add small ridge to avoid singularity
    ESS_reg = ESS + eps * np.eye(K)

    # μ = (ESS^-1 ES^T X)^T  -> use solve instead of explicit inverse
    mu = np.linalg.solve(ESS_reg, ES.T @ X).T   # (D,K)

    # σ update (safe version)
    num = (
        np.trace(X.T @ X) +
        np.trace(mu.T @ mu @ ESS) -
        2.0 * np.trace(ES.T @ X @ mu)
    )
    sigma2 = max(num / (N * D), 1e-12)
    sigma = float(np.sqrt(sigma2))

    # π update
    pi = np.clip(ES.mean(axis=0), 0.05, 0.95)   # (K,)

    return mu, sigma, pi

## Combined EM Function

In [76]:
import numpy as np

def LearnBinFactors(X, K, iterations=50, eps_F=1e-6):
    """
    mu, sigma2, pi, F_hist = LearnBinFactors(X,K,iterations)

    X          : (N,D) data
    K          : number of binary factors
    iterations : max EM iterations
    eps_F      : convergence threshold on |F_t - F_{t-1}|
    """
    N, D = X.shape

    # ----- initialise parameters -----
    rng = np.random.default_rng()
    mu = rng.normal(0, 1, size=(D, K))
    sigma2 = 1.0
    pi = np.full(K, 0.5)

    # initial variational parameters λ
    lam = rng.uniform(0.25, 0.75, size=(N, K))

    F_hist = []

    for it in range(iterations):
        # ===== E-step: mean-field over s, holding θ fixed =====
        lam, F_vals = MeanField(
            X, mu.T, sigma2, pi, lam,
            maxsteps=100, eps=1e-6
        )
        F_E = F_vals[-1]      # free energy at end of inner loop

        # ===== M-step: update μ, σ, π using current expectations =====
        ES = lam                     # (N,K) = E_q[s]
        # E_q[ss'] factorises: diag = λ_k, off-diag = λ_k λ_{k'}
        ESS = np.zeros((K, K))
        ESS += ES.T @ ES             # includes λ_k λ_{k'}, and λ_k^2 on diag

        mu, sigma, pi_row = MStep(X, ES, ESS)
        mu = mu                      # (D,K)
        sigma2 = sigma**2
        pi = pi_row.ravel()

        # recompute ELBO with updated θ as a debugging check (optional)
        # here we reuse F_E as our monitored quantity
        F_hist.append(F_E)

        # check monotonic increase
        if it > 0 and F_hist[-1] < F_hist[-2] - 1e-8:
            print(f"Warning: F decreased at EM iter {it}: "
                  f"{F_hist[-2]:.6f} -> {F_hist[-1]:.6f}")

        # convergence on F
        if it > 0 and abs(F_hist[-1] - F_hist[-2]) < eps_F:
            break

    return mu, sigma2, pi, np.array(F_hist)

## Run on genimages.py

In [64]:
# Load the data
%run /work/genimages.py
X = Y     # shape (N, 16)
N, D = X.shape                   # D should be 16
K = 8                            # number of latent factors

In [209]:
mu, sigma2, pi, F_hist = LearnBinFactors(X, K=8, iterations=50)

print("F_hist (first 5):", F_hist[:5])
print("mu range:", mu.min(), mu.max())
print("sigma2:", sigma2)
print("pi:", pi)

for k in range(8):
    plt.subplot(2,4,k+1)
    plt.imshow(mu[:,k].reshape(4,4), cmap="gray")
    plt.axis("off")
plt.show()

## N=1

In [210]:
# Extract the first datapoint
x1 = Y[0:1, :]      # shape (1,D)
N1, D = x1.shape
K = mu.shape[1]     # (D,K)

# initial λ for this point 
lam0 = np.full((1, K), 0.5)

# Different sigmas
sigmas = [0.1, 1.0, 5.0]
F_hist_all = []

for sigma in sigmas:
    sigma2 = sigma**2
    lam, F_hist = MeanField(
        x1,           # X with N=1
        mu.T,         # (K,D) inside function
        sigma2,
        pi,
        lam0,
        maxsteps=100,
        eps=1e-10     # tight so you see more iterations
    )
    F_hist_all.append(F_hist)

plt.figure(figsize=(10,4))

# F(t)
plt.subplot(1,2,1)
for sigma, F_hist in zip(sigmas, F_hist_all):
    plt.plot(F_hist, label=f"σ={sigma}")
plt.xlabel("Iteration t")
plt.ylabel("F")
plt.title("Free energy vs iteration (N=1)")
plt.legend()
plt.grid(True)

# log(F(t)-F(t-1))
plt.subplot(1,2,2)
for sigma, F_hist in zip(sigmas, F_hist_all):
    F_diff = np.diff(F_hist)
    # keep positive diffs; clip very small negatives due to numerics
    F_diff = np.clip(F_diff, 1e-15, None)
    plt.plot(np.log(F_diff), label=f"σ={sigma}")
plt.xlabel("Iteration t")
plt.ylabel("log(F_t - F_{t-1})")
plt.title("Convergence rate (log increment)")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Variational Bayes for binary factors

##  Hyperparameter optimisation algorithm

In [100]:
def ard(X, K_max, max_iters=200, tol=1e-4, a0=1e-3, b0=1e-3):
    N, D = X.shape
    K = K_max

    lam = np.clip(0.5 + 0.1*np.random.randn(N, K), 1e-3, 1-1e-3)
    m   = 0.1 * np.random.randn(K, D)          # rows m_k
    sigma2 = np.var(X)
    pi  = 0.3 * np.ones(K)

    # ARD hyperparameters: q(α_k) = Gamma(a_k, b_k)
    a = a0 * np.ones(K)
    b = b0 * np.ones(K)

    F_hist, Keff_hist = [], []

    for t in range(max_iters):
        # ===== E-step: update λ =====
        mu_kd = m.copy()                       # K×D
        for n in range(N):
            x_n = X[n]
            for k in range(K):
                lam_minus = lam[n].copy(); lam_minus[k] = 0.0
                E_mu_minus = lam_minus @ mu_kd
                resid = x_n - E_mu_minus
                mu_k = mu_kd[k]
                log_prior_odds = np.log(pi[k] / (1-pi[k]))
                llr = (resid @ mu_k) / sigma2 - 0.5 * (mu_k @ mu_k) / sigma2
                logit = np.clip(log_prior_odds + llr, -20, 20)
                lam[n, k] = 1.0 / (1.0 + np.exp(-logit))

        # ===== M-step: q(w_d) for all d, then collect moments for each k =====
        # we’ll store posterior means/covariances of w_d
        mu_wd = np.zeros((D, K))
        Sigma_wd_diag = np.zeros((D, K))       # only diag needed for ARD

        alpha = a / b                          # E[α_k]
        A = np.diag(alpha)                     # K×K

        ES  = lam                              # N×K
        ESS = ES.T @ ES                        # K×K

        for d in range(D):
            x_d = X[:, d]                      # N
            # Σ_wd^{-1} = A + (1/σ²) E[ssᵀ]
            Sigma_inv = A + ESS / sigma2       # K×K
            Sigma = np.linalg.inv(Sigma_inv)
            # μ_wd = Σ (1/σ²) E[s]ᵀ x_d
            mu = Sigma @ (ES.T @ x_d) / sigma2 # K
            mu_wd[d] = mu
            Sigma_wd_diag[d] = np.diag(Sigma)

        # stack rows m_k: k-th row collects wd,k over d
        m = mu_wd.T                            # K×D

        # ===== Hyper-M: update q(α_k) =====
        # a_k = a0 + (D/2)
        # b_k = b0 + 0.5 * Σ_d E[w_{d,k}²]
        Ew2 = (m**2 + Sigma_wd_diag.T).sum(axis=1)    # length K
        a = a0 + 0.5 * D
        b = b0 + 0.5 * Ew2
        alpha = a / b

        # ===== Update π from λ =====
        pi = np.clip(lam.mean(axis=0), 1e-3, 1-1e-3)

        # ===== (optional) σ² update using current m, λ =====
        X_hat = lam @ m                        # N×D
        sigma2 = np.mean((X - X_hat)**2)

        # ===== Track ELBO & K_eff =====
        lam_tmp, F_vals = MeanField(X, m, sigma2, pi, lam, maxsteps=1, eps=0.0)
        F_t = F_vals[-1]
        F_hist.append(F_t)
        
        Keff = np.sum(alpha < 10.0)            # e.g. “active” if E[α_k] < 10
        Keff_hist.append(Keff)

        if t > 5 and abs(F_hist[-1] - F_hist[-2]) < tol:
            break

    params = dict(lambda_=lam, m=m, alpha=alpha, pi=pi, sigma2=sigma2)
    return np.array(F_hist), np.array(Keff_hist), params

## Results

In [12]:
K_values = [2, 4, 5, 8, 10, 11, 13, 15, 16, 18, 20]

results = {}
for K in K_values:
    F_hist, Keff_hist, params = ard(X, K_max=K, max_iters=200, tol=1e-4)
    results[K] = dict(F=np.asarray(F_hist),
                      Keff=np.asarray(Keff_hist),
                      params=params)

# ================================
# ELBO / Keff vs iteration (plot only a few representative Ks)
# ================================
K_values = [2, 5, 8, 15, 20]
colors = {
    2: "purple",
    5: "tab:blue",
    8: "tab:red",
    15: "tab:green",
    20: "tab:orange"
}

plt.figure(figsize=(10, 4))

# ----- ELBO vs iteration -----
plt.subplot(1, 2, 1)
for K in K_values:
    F = results[K]["F"]
    style = "-" if K == 8 else "--"  # highlight K=10
    plt.plot(F, linestyle=style, color=colors[K], label=f"K={K}")
plt.xlabel("VB iteration")
plt.ylabel("Free energy F")
plt.title("VB free energy vs iteration")
plt.grid(True, alpha=0.3)
plt.legend(frameon=False)

# ----- Effective K vs iteration -----
plt.subplot(1, 2, 2)
for K in K_values:
    Keff = results[K]["Keff"]
    style = "-" if K == 8 else "--"
    plt.plot(Keff, linestyle=style, color=colors[K], label=f"K={K}")
plt.xlabel("VB iteration")
plt.ylabel("Effective K")
plt.title("Effective number of factors vs iteration")
plt.grid(True, alpha=0.3)
plt.legend(frameon=False)

plt.tight_layout()
plt.show()

# ================================
# Final F and Keff vs initial K
# ================================
final_F = []
final_Keff = []
for K in K_values:
    final_F.append(results[K]["F"][-1])
    final_Keff.append(results[K]["Keff"][-1])

plt.figure(figsize=(8, 3))

plt.subplot(1, 2, 1)
plt.plot(K_values, final_F, "o-")
plt.xlabel("Initial K")
plt.ylabel("Final free energy F")
plt.title("Final ELBO vs initial K")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(K_values, final_Keff, "o-")
plt.xlabel("Initial K")
plt.ylabel("Final effective K")
plt.title("Final effective K vs initial K")
plt.grid(True)

plt.tight_layout()
plt.show()

# EP

In [133]:
def EP_single(x, mu, sigma2, pi, max_iter=50, tol=1e-4, eps=1e-9):
    x = np.asarray(x)
    K, D = mu.shape

    logit_pi = np.log(pi) - np.log(1 - pi)

    eta_site = np.zeros(K)
    eta_post = logit_pi + eta_site
    lam = expit(eta_post)

    for it in range(max_iter):
        lam_old = lam.copy()

        for k in range(K):
            eta_cav = eta_post[k] - eta_site[k]
            lam_cav = expit(eta_cav)

            s_mean_others = lam.copy()
            s_mean_others[k] = 0.0
            mean_contrib_others = s_mean_others @ mu
            mu_k = mu[k]

            m0 = mean_contrib_others
            m1 = mean_contrib_others + mu_k

            ll0 = -0.5 / sigma2 * np.sum((x - m0) ** 2)
            ll1 = -0.5 / sigma2 * np.sum((x - m1) ** 2)

            log_cav0 = 0.0
            log_cav1 = eta_cav

            log_p0 = log_cav0 + ll0
            log_p1 = log_cav1 + ll1
            mmax = max(log_p0, log_p1)
            p0 = np.exp(log_p0 - mmax)
            p1 = np.exp(log_p1 - mmax)
            Z = p0 + p1

            lam_tilt = p1 / Z

            # clamp to avoid 0 or 1
            lam_tilt = np.clip(lam_tilt, eps, 1 - eps)

            eta_post_new_k = np.log(lam_tilt) - np.log(1 - lam_tilt)
            eta_site[k] = eta_post_new_k - eta_cav

        eta_post = logit_pi + eta_site
        lam = expit(eta_post)

        if np.max(np.abs(lam - lam_old)) < tol:
            break

    return lam

In [136]:
def EP(X, mu, sigma2, pi, max_iter_ep=50, tol=1e-4):
    N, D = X.shape
    K = mu.shape[0]
    lam_all = np.zeros((N, K))
    for n in range(N):
        lam_all[n] = EP_single(X[n], mu, sigma2, pi,
                                         max_iter=max_iter_ep, tol=tol)
    return lam_all

## Loopy BP

In [112]:
def loopyBP_single(x, mu, sigma2, pi, max_iter=50, tol=1e-4):
    """
    Approximate loopy BP for p(s | x) in the binary factor model.

    x   : (D,)
    mu  : (K,D) rows mu_k
    pi  : (K,)
    """
    x = np.asarray(x)
    K, D = mu.shape

    # log-odds messages from prior and likelihood to each s_k
    logit_prior = np.log(pi) - np.log(1-pi)
    # initialise variable beliefs
    logit_bel = logit_prior.copy()
    lam = expit(logit_bel)

    for it in range(max_iter):
        lam_old = lam.copy()

        # "factor → variable" messages from the (approximated) likelihood
        logit_like = np.zeros(K)

        # mean-field over others to make factor update tractable
        mean_others = lam @ mu          # (D,)

        for k in range(K):
            mu_k = mu[k]

            # remove contribution of s_k from mean_others
            mean_minus = mean_others - lam[k] * mu_k

            # Likelihood for s_k = 0 and 1 using Gaussian with current mean field:
            m0 = mean_minus
            m1 = mean_minus + mu_k

            ll0 = -0.5/sigma2 * np.sum((x - m0)**2)
            ll1 = -0.5/sigma2 * np.sum((x - m1)**2)

            logit_like[k] = ll1 - ll0    # factor→variable log-odds

        # variable beliefs: prior ⊕ all factor messages
        logit_bel = logit_prior + logit_like
        logit_bel = np.clip(logit_bel, -20, 20)
        lam = expit(logit_bel)

        if np.max(np.abs(lam - lam_old)) < tol:
            break

    return lam   # approximate marginals q(s_k=1|x)

In [115]:
def loopyBP(X, mu, sigma2, pi, max_iter=50, tol=1e-4):
    N, D = X.shape
    K = mu.shape[0]
    lam_all = np.zeros((N, K))
    for n in range(N):
        lam_all[n] = loopyBP_single(X[n], mu, sigma2, pi,
                                               max_iter=max_iter, tol=tol)
    return lam_all

## VB vs EP vs BP

In [142]:
# Learn theta
mu_vb, sigma2_vb, pi_vb, F_vb = LearnBinFactors(X, K=8, iterations=100)

# VB mean-field λ
lam_vb, _ = MeanField(X, mu_vb.T, sigma2_vb, pi_vb, 
                      lambda0=0.5*np.ones((N,8)), maxsteps=50)

# Reconstruction errors per datapoint under VB
X_hat_vb = lam_vb @ mu_vb.T          # (N,16)
err_per_x = np.mean((X - X_hat_vb)**2, axis=1)

# Choose indices
clean_idx = np.argmin(err_per_x)         # best reconstructed
noisy_idx = np.argmax(err_per_x)         # worst reconstructed
indices = [clean_idx, noisy_idx]

# EP λ
lam_ep = EP(X, mu_vb.T, sigma2_vb, pi_vb)

# Loopy BP λ
lam_bp = loopyBP(X, mu_vb.T, sigma2_vb, pi_vb)

K = 8

# Average squared reconstruction error
def recon_error(X, lam, mu):
    X_hat = lam @ mu.T
    return np.mean((X - X_hat)**2, axis=1)

err_vb = recon_error(X, lam_vb, mu_vb)
err_ep = recon_error(X, lam_ep, mu_vb)
err_bp = recon_error(X, lam_bp, mu_vb)

for n in indices:
    x = X[n].reshape(4,4)

    # reconstructions
    x_vb = (lam_vb[n] @ mu_vb.T).reshape(4,4)
    x_ep = (lam_ep[n] @ mu_vb.T).reshape(4,4)
    x_bp = (lam_bp[n] @ mu_vb.T).reshape(4,4)
    
    # label best / worst
    if n == clean_idx:
        rec_type = "best reconstruction"
    elif n == noisy_idx:
        rec_type = "worst reconstruction"
    else:
        rec_type = ""

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 5, 1)
    width = 0.3
    ks = np.arange(K)
    plt.bar(ks - width, lam_vb[n], width, alpha=0.7, label="VB")
    plt.bar(ks,         lam_ep[n], width, alpha=0.7, label="EP")
    plt.bar(ks + width, lam_bp[n], width, alpha=0.7, label="BP")
    plt.xticks(ks)
    plt.ylim(0, 1)

    title_main = f"λ, n={n}"
    title_errs = (
        f"err(VB)={err_vb[n]:.3e}, "
        f"err(EP)={err_ep[n]:.3e}, "
        f"err(BP)={err_bp[n]:.3e}"
    )
    if rec_type:
        plt.title(f"{title_main} ({rec_type})\n{title_errs}")
    else:
        plt.title(f"{title_main}\n{title_errs}")

    plt.legend()

    plt.subplot(1, 5, 2)
    plt.imshow(x, cmap="gray")
    plt.axis("off")
    plt.title("Original")

    plt.subplot(1, 5, 3)
    plt.imshow(x_vb, cmap="gray")
    plt.axis("off")
    plt.title("VB")

    plt.subplot(1, 5, 4)
    plt.imshow(x_ep, cmap="gray")
    plt.axis("off")
    plt.title("EP")

    plt.subplot(1, 5, 5)
    plt.imshow(x_bp, cmap="gray")
    plt.axis("off")
    plt.title("BP")

    plt.tight_layout()
    plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cb182644-878e-48cb-992b-68a78a5afe3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>